<a href="https://colab.research.google.com/github/vitorchaud/calc_num/blob/main/decomposi%C3%A7%C3%A3o_LU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cálculo Numérico para a Engenheria Elétrica (CALNE) 2S - 2025**

---

# **Método para decomposição LU**
---



In [3]:
import pandas as pd

def decomposicao_LU_e_resolve(matriz_aumentada):
    """
    Executa a Decomposição LU de uma matriz (com sua coluna de termos constantes),
    em seguida resolve o sistema linear via L e U.

    Parâmetros:
        matriz_aumentada (lista de listas de floats):
            matriz de tamanho n x (n+1).
            As primeiras n colunas compõem a matriz A,
            e a última coluna é o vetor b.

    Retorna:
        L, U, x
        - L: matriz de fatores (n x n)
        - U: matriz triangular superior (n x n)
        - x: solução do sistema (lista de tamanho n)
    """

    print('\n== Decomposição LU ==')

    # Tamanho do sistema (n x n) + 1 coluna de termos
    n = len(matriz_aumentada)

    # Separamos A (n x n) e b (n x 1) para didática
    # (Embora possamos trabalhar "in place", fica mais claro separar)
    A = [[matriz_aumentada[i][j] for j in range(n)] for i in range(n)]
    b = [matriz_aumentada[i][n] for i in range(n)]

    print('\n-> Matriz A inicial:')
    print(pd.DataFrame(A))

    print('\n-> Vetor b inicial:')
    print(b)

    # Inicia L como identidade e U como cópia de A
    L = [[0.0]*n for _ in range(n)]
    for i in range(n):
        L[i][i] = 1.0  # diagonal = 1

    U = [[A[i][j] for j in range(n)] for i in range(n)]  # cópia de A

    print('\n-> Iniciando L como identidade:')
    print(pd.DataFrame(L))

    print('\n-> Iniciando U como cópia de A:')
    print(pd.DataFrame(U))

    # === Fase de Eliminação para frente ===
    print('\n== Eliminação para frente (formando U e registrando fatores em L) ==')

    for i in range(n):
        # 1) Se o pivô for zero, tentamos trocar com alguma linha abaixo
        if U[i][i] == 0:
            for r in range(i+1, n):
                if U[r][i] != 0:
                    # troca linhas de U
                    U[i], U[r] = U[r], U[i]
                    # troca b (para coerência na hora de resolver depois)
                    b[i], b[r] = b[r], b[i]
                    # troca as partes correspondentes em L (até a coluna i-1)
                    # para não estragar os fatores que já foram computados
                    for c in range(i):
                        L[i][c], L[r][c] = L[r][c], L[i][c]
                    print(f'\n-> Pivô zero detectado na linha {i}, trocando com linha {r}...')
                    break
            else:
                raise ValueError("A matriz é singular ou não possui solução única.")

        pivot = U[i][i]
        if pivot == 0:
            raise ValueError("Não foi possível encontrar pivô não-nulo (matriz singular).")

        # 2) Elimina abaixo do pivô (linha i), para as linhas j > i
        for j in range(i+1, n):
            fator = U[j][i] / pivot
            L[j][i] = fator  # salva o fator em L
            print(f'\n-> Zerando elemento U[{j}][{i}] usando fator {fator:.4f}')

            # atualiza linha j de U
            for k in range(i, n):
                U[j][k] = U[j][k] - fator * U[i][k]

            print(f'U (parcial) após zerar linha {j}:')
            print(pd.DataFrame(U))

    print('\n-> Matriz L obtida:')
    print(pd.DataFrame(L))

    print('\n-> Matriz U obtida:')
    print(pd.DataFrame(U))

    # === Resolvendo o sistema via L e U ===
    # Precisamos resolver L*y = b, depois U*x = y

    print('\n== Resolvendo L * y = b (Substituição para frente) ==')

    # Inicializa y com zeros
    y = [0.0 for _ in range(n)]
    for i in range(n):
        soma = 0.0
        for k in range(i):
            soma += L[i][k] * y[k]
        y[i] = (b[i] - soma) / L[i][i]  # L[i][i] deve ser 1.0 se for LU sem pivot “mais elaborado”
        print(f'y[{i}] = (b[{i}] - {soma}) / {L[i][i]} = {y[i]}')

    print('\n== Resolvendo U * x = y (Substituição para trás) ==')
    x = [0.0 for _ in range(n)]
    for i in range(n-1, -1, -1):
        soma = 0.0
        for k in range(i+1, n):
            soma += U[i][k] * x[k]
        x[i] = (y[i] - soma) / U[i][i]
        print(f'x[{i}] = (y[{i}] - {soma}) / {U[i][i]} = {x[i]}')

    print('\n== Resultado Final ==')
    print('Matriz L:')
    print(pd.DataFrame(L))
    print('\nMatriz U:')
    print(pd.DataFrame(U))

    return L, U, x


# ================ EXEMPLO DE USO ================
if __name__ == "__main__":
    matriz_aumentada = [
        [2,   1, -1,   8],
        [-3, -1,  2, -11],
        [-2,  1,  2,  -3]
    ]

    L, U, solucao = decomposicao_LU_e_resolve(matriz_aumentada)
    print("\nSOLUÇÃO FINAL DO SISTEMA:", solucao)



== Decomposição LU ==

-> Matriz A inicial:
   0  1  2
0  2  1 -1
1 -3 -1  2
2 -2  1  2

-> Vetor b inicial:
[8, -11, -3]

-> Iniciando L como identidade:
     0    1    2
0  1.0  0.0  0.0
1  0.0  1.0  0.0
2  0.0  0.0  1.0

-> Iniciando U como cópia de A:
   0  1  2
0  2  1 -1
1 -3 -1  2
2 -2  1  2

== Eliminação para frente (formando U e registrando fatores em L) ==

-> Zerando elemento U[1][0] usando fator -1.5000
U (parcial) após zerar linha 1:
     0    1    2
0  2.0  1.0 -1.0
1  0.0  0.5  0.5
2 -2.0  1.0  2.0

-> Zerando elemento U[2][0] usando fator -1.0000
U (parcial) após zerar linha 2:
     0    1    2
0  2.0  1.0 -1.0
1  0.0  0.5  0.5
2  0.0  2.0  1.0

-> Zerando elemento U[2][1] usando fator 4.0000
U (parcial) após zerar linha 2:
     0    1    2
0  2.0  1.0 -1.0
1  0.0  0.5  0.5
2  0.0  0.0 -1.0

-> Matriz L obtida:
     0    1    2
0  1.0  0.0  0.0
1 -1.5  1.0  0.0
2 -1.0  4.0  1.0

-> Matriz U obtida:
     0    1    2
0  2.0  1.0 -1.0
1  0.0  0.5  0.5
2  0.0  0.0 -1.0

=